In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from itertools import permutations

import numpy as np

In [2]:
PROBLEM_SIZE = 20
NUM_SETS = 40 #number of tiles
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

print(f"NUM_SETS: {len(SETS)}, {SETS}")

NUM_SETS: 40, (array([False, False, False, False,  True, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False]), array([False, False,  True, False, False, False, False,  True, False,
       False, False,  True,  True, False, False, False, False, False,
       False,  True]), array([ True, False, False,  True,  True, False, False,  True, False,
        True, False,  True,  True, False,  True, False, False, False,
        True, False]), array([False, False, False, False, False, False, False,  True, False,
       False, False, False,  True, False, False, False, False, False,
       False, False]), array([False, False,  True, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), array([ True, False,  True, False,  True, False, False, False,  True,
       False, False,  True,  True,  True,  True,  True, False, False,
       False, False]), array([

In [3]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

## REMOVING DUPLICATED SETS (OPTIONAL)

In [12]:
#REMOVING DUPLICATED SETS MAKES SENSE ONLY WHEN PROBLEM_SIZE IS LITTLE OR PROBLEM_SIZE << NUM_SETS

sets_list = list(SETS)

unique_sets = set(tuple(arr) for arr in sets_list)

unique_sets_list = [np.array(arr) for arr in unique_sets]

unique_sets_tuple = tuple(unique_sets_list)

SETS = unique_sets_tuple

print(f"NUM_SETS: {len(SETS)}, {SETS}")

NUM_SETS: 40, (array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False,  True, False,  True,
        True,  True]), array([False, False, False, False,  True, False, False,  True, False,
        True, False, False, False,  True, False, False, False, False,
       False, False]), array([False, False,  True, False, False, False, False, False,  True,
        True,  True, False, False,  True, False, False,  True, False,
       False, False]), array([False, False, False, False, False, False, False,  True,  True,
       False, False, False,  True, False, False,  True, False,  True,
        True, False]), array([ True, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False,  True, False, False,
       False,  True]), array([False, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False,  True,  True, False,
       False, False]), array([

## MY A*

In [ ]:
def check_num(min_tiles, state):
    for i in permutations(state.not_taken, r = min_tiles):
        new_state = State(state.taken, state.not_taken)
        for j in i:
            new_state = State(new_state.taken ^ {j}, new_state.not_taken ^ {j})

        if goal_check(new_state):
            return True

    return False

In [4]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action}, current_state.not_taken ^ {action}
        )

        #COST (g) = number of tiles taken starting from the root.
        #HEURISTIC (h) = min number of tiles I need to cover all the sets. This number is <= than the distance.

        min_tiles = 0

        while min_tiles != distance(new_state): 
            min_tiles += 1

            if check_num(min_tiles, new_state):
                break


        frontier.put((len(new_state.taken) + min_tiles, new_state))
        print(f"g={len(new_state.taken)} , distance = {distance(new_state)},  h={min_tiles} , sum={len(new_state.taken) + min_tiles}, {new_state}")
    _, current_state = frontier.get()
    print(f"STEP {counter}:  {current_state}")

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

g=1 , distance = 18,  h=3 , sum=4, State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
g=1 , distance = 15,  h=3 , sum=4, State(taken={1}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
g=1 , distance = 11,  h=3 , sum=4, State(taken={2}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
g=1 , distance = 18,  h=3 , sum=4, State(taken={3}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
g=1 , distance = 18,  h=4 , sum=5, State(taken={4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26

In [5]:
current_state

State(taken={16, 10, 28, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})

In [6]:
goal_check(current_state)

True

## PROF A*

In [7]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def f(state):
    return len(state.taken) + h3(state)

In [8]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
#with tqdm(total=None) as pbar:
while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
            print(f"c+h = {f(new_state)}, {new_state}")
        _, current_state = frontier.get()
        print(f"STEP {counter}: {current_state}")
        #pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

c+h = 3, State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
c+h = 3, State(taken={1}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
c+h = 3, State(taken={2}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
c+h = 3, State(taken={3}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
c+h = 3, State(taken={4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39})
c+h = 4, State(taken={5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12,

In [9]:
current_state

State(taken={8, 26, 14, 39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38})

In [10]:
goal_check(current_state)

True